In [ ]:
import pickle
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse as sp
from mpl_toolkits.basemap import Basemap
from IPython import display
from scipy.spatial import ConvexHull, Delaunay
%matplotlib inline

In [ ]:
# Load in our pretty pandas database
with open("../data/modis.pkl") as fpd:
    modis = cPickle.load(fpd)
print "loaded pandas database with %d points" %len(modis)
ak_fires = modis[(modis.lat < 71) & (modis.lat > 55) & (modis.long > -165) & (modis.long < -138)]
ak_summer_fires = ak_fires[(ak_fires.year == 2013) & (ak_fires.month >= 6) & (ak_fires.month <= 8)]
ak_fires_2013 = ak_fires[ak_fires.year == 2013]

In [ ]:
deg_thresh = .05
N = len(ak_fires_2013)
thresh_graph = sp.lil_matrix((N,N))
lat_long_mat = np.transpose(np.array((np.array(ak_fires_2013.lat),np.array(ak_fires_2013.long))))
print "lat_long_mat shape: " + str(lat_long_mat.shape)
for i in xrange(N):
    p_i = np.array([ak_fires_2013.iloc[i].lat, ak_fires_2013.iloc[i].long])
    dist_arr = np.linalg.norm(lat_long_mat - p_i, axis=1)
    thresh_arr = dist_arr < deg_thresh
    thresh_graph[i,:] = thresh_arr
    if i % 1000 == 0:
        print "finished %d of %d" %(i, N)
n_CCs, CCs = sp.csgraph.connected_components(thresh_graph, directed=False)
print "%d connected components" %(n_CCs)

In [ ]:
# Kill singleton clusters
valid_n_CCs = 0
valid_CCs = np.zeros(N) + np.inf
for clust in xrange(n_CCs):
    if len(np.where(CCs == clust)[0]) > 1:
        valid_CCs[np.where(CCs == clust)] = valid_n_CCs
        valid_n_CCs += 1
print "Found %d clusts" % valid_n_CCs

In [ ]:
def monthday2day(month, day, leapyear=False):
    """Convert month/day into days since Jan 1"""
    if leapyear:
        month_arr = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    else:
        month_arr = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    days = 0
    for mon in xrange(1, month):
        days += month_arr[mon - 1]
    days += day - 1
    return days

ep_sep_arr = np.zeros(n_CCs)  # Number of time-separated episodes per fire
ep_sep_len = 2  # Number of days without fire before we declare a new episode


for clust in xrange(n_CCs):
    clust_fires = ak_fires_2013.iloc[np.where(CCs == clust)]
    time_arr = np.zeros(len(clust_fires))
    for i,(month,day) in enumerate(zip(clust_fires.month, clust_fires.day)):
        my_day = monthday2day(month, day, leapyear=False)
        time_arr[i] = my_day
    sorted_times = np.sort(time_arr)
    max_day = int(sorted_times[-1])
    min_day = int(sorted_times[0])
    time_freq_arr = np.zeros(max_day-min_day+1)
    for month,day in zip(clust_fires.month, clust_fires.day):
        my_day = monthday2day(month, day, leapyear=False)
        time_freq_arr[my_day - min_day] += 1
    display.clear_output(wait=True)
    plt.plot(time_freq_arr, 'r-')
plt.show()